In [11]:
import torch
import torchvision
import torchvision.transforms as transforms

# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 下载数据集
trainset = torchvision.datasets.FashionMNIST('./data', download=True, train=True, transform=transform)
testset = torchvision.datasets.FashionMNIST('./data', download=True, train=False, transform=transform)

# 数据加载器
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)


In [12]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [13]:
import torch.optim as optim

net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [14]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/fashion_mnist_experiment_1')


In [15]:
# 记录训练过程
num_epochs = 3
for epoch in tqdm(range(num_epochs)):
    running_loss = 0.0
    correct = 0
    total = 0

    # 训练阶段
    net.train()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        # 前向传播 + 后向传播 + 优化
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 统计数据
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # 每100步记录一次损失和准确率
        if i % 100 == 99:
            print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}")
            writer.add_scalar('training_loss', running_loss / 100, epoch * len(trainloader) + i)
            writer.add_scalar('training_accuracy', 100 * correct / total, epoch * len(trainloader) + i)
            running_loss = 0.0

    # 记录训练损失和准确率
    epoch_loss = running_loss / len(trainloader)
    epoch_accuracy = 100 * correct / total
    writer.add_scalar('epoch_loss', epoch_loss, epoch)
    writer.add_scalar('epoch_accuracy', epoch_accuracy, epoch)

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # 记录模型的权重和梯度分布
    writer.add_histogram('conv1_weights', net.conv1.weight, epoch)
    writer.add_histogram('conv1_gradients', net.conv1.weight.grad, epoch)

    # 记录模型计算图（第一次进行前向传播时）
    if epoch == 0:
        writer.add_graph(net, inputs)

    # 记录部分训练图像
    if epoch == 0:  # 仅在第一个 epoch 时记录
        img_grid = torchvision.utils.make_grid(inputs)
        writer.add_image('training_images', img_grid)


  0%|          | 0/3 [00:00<?, ?it/s]

[1,   100] loss: 2.298
[1,   200] loss: 2.292
[1,   300] loss: 2.277
[1,   400] loss: 2.250
[1,   500] loss: 2.123
[1,   600] loss: 1.703
[1,   700] loss: 1.342
[1,   800] loss: 1.227
[1,   900] loss: 1.114
[1,  1000] loss: 1.130
[1,  1100] loss: 1.037
[1,  1200] loss: 0.913
[1,  1300] loss: 0.893
[1,  1400] loss: 0.883
[1,  1500] loss: 0.859
[1,  1600] loss: 0.917
[1,  1700] loss: 0.817
[1,  1800] loss: 0.822
[1,  1900] loss: 0.790
[1,  2000] loss: 0.771
[1,  2100] loss: 0.738
[1,  2200] loss: 0.812
[1,  2300] loss: 0.678
[1,  2400] loss: 0.828
[1,  2500] loss: 0.689
[1,  2600] loss: 0.758
[1,  2700] loss: 0.697
[1,  2800] loss: 0.676
[1,  2900] loss: 0.670
[1,  3000] loss: 0.692
[1,  3100] loss: 0.701
[1,  3200] loss: 0.691
[1,  3300] loss: 0.650
[1,  3400] loss: 0.705
[1,  3500] loss: 0.625
[1,  3600] loss: 0.594
[1,  3700] loss: 0.609
[1,  3800] loss: 0.607
[1,  3900] loss: 0.710
[1,  4000] loss: 0.641
[1,  4100] loss: 0.657
[1,  4200] loss: 0.648
[1,  4300] loss: 0.555
[1,  4400] 

 33%|███▎      | 1/3 [00:48<01:37, 48.81s/it]

Epoch [1/3] - Loss: 0.0000, Accuracy: 76.69%
[2,   100] loss: 0.459
[2,   200] loss: 0.387
[2,   300] loss: 0.485
[2,   400] loss: 0.386
[2,   500] loss: 0.333
[2,   600] loss: 0.470
[2,   700] loss: 0.424
[2,   800] loss: 0.447
[2,   900] loss: 0.431
[2,  1000] loss: 0.378
[2,  1100] loss: 0.394
[2,  1200] loss: 0.394
[2,  1300] loss: 0.442
[2,  1400] loss: 0.418
[2,  1500] loss: 0.400
[2,  1600] loss: 0.389
[2,  1700] loss: 0.347
[2,  1800] loss: 0.403
[2,  1900] loss: 0.412
[2,  2000] loss: 0.421
[2,  2100] loss: 0.378
[2,  2200] loss: 0.349
[2,  2300] loss: 0.422
[2,  2400] loss: 0.333
[2,  2500] loss: 0.371
[2,  2600] loss: 0.419
[2,  2700] loss: 0.381
[2,  2800] loss: 0.398
[2,  2900] loss: 0.411
[2,  3000] loss: 0.402
[2,  3100] loss: 0.372
[2,  3200] loss: 0.381
[2,  3300] loss: 0.374
[2,  3400] loss: 0.476
[2,  3500] loss: 0.419
[2,  3600] loss: 0.383
[2,  3700] loss: 0.331
[2,  3800] loss: 0.421
[2,  3900] loss: 0.363
[2,  4000] loss: 0.464
[2,  4100] loss: 0.354
[2,  4200] l

 67%|██████▋   | 2/3 [01:24<00:41, 41.29s/it]

Epoch [2/3] - Loss: 0.0000, Accuracy: 86.51%
[3,   100] loss: 0.367
[3,   200] loss: 0.400
[3,   300] loss: 0.283
[3,   400] loss: 0.282
[3,   500] loss: 0.315
[3,   600] loss: 0.355
[3,   700] loss: 0.341
[3,   800] loss: 0.411
[3,   900] loss: 0.403
[3,  1000] loss: 0.304
[3,  1100] loss: 0.297
[3,  1200] loss: 0.333
[3,  1300] loss: 0.301
[3,  1400] loss: 0.300
[3,  1500] loss: 0.331
[3,  1600] loss: 0.312
[3,  1700] loss: 0.283
[3,  1800] loss: 0.299
[3,  1900] loss: 0.299
[3,  2000] loss: 0.282
[3,  2100] loss: 0.357
[3,  2200] loss: 0.292
[3,  2300] loss: 0.376
[3,  2400] loss: 0.334
[3,  2500] loss: 0.299
[3,  2600] loss: 0.340
[3,  2700] loss: 0.320
[3,  2800] loss: 0.310
[3,  2900] loss: 0.262
[3,  3000] loss: 0.368
[3,  3100] loss: 0.288
[3,  3200] loss: 0.390
[3,  3300] loss: 0.301
[3,  3400] loss: 0.327
[3,  3500] loss: 0.312
[3,  3600] loss: 0.268
[3,  3700] loss: 0.315
[3,  3800] loss: 0.331
[3,  3900] loss: 0.273
[3,  4000] loss: 0.262
[3,  4100] loss: 0.345
[3,  4200] l

100%|██████████| 3/3 [02:06<00:00, 41.68s/it]

[3, 15000] loss: 0.326
Epoch [3/3] - Loss: 0.0000, Accuracy: 88.07%


100%|██████████| 3/3 [02:06<00:00, 42.33s/it]


Test Accuracy: 86.49%


In [16]:
# 测试阶段
correct = 0
total = 0
net.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")
writer.add_scalar('test_accuracy', test_accuracy)

# 关闭 TensorBoard 记录器
writer.close()

Test Accuracy: 86.49%
